# GRNet implementation

## Imports 

In [1]:
import numpy as np
from tensorflow.keras.models import load_model
from os.path import join
import pickle
import time
import os
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import tensorflow.keras.backend as K
from typing import Union
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras.initializers import Constant
from random import random

## Custom Functions

In [2]:
def custom_multilabel_loss_v3(y_true, y_pred):
    # Fix tensors structure
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)

    multiclass_squared_error = tf.math.reduce_sum(K.square(y_true - y_pred), axis=-1)
    label_value = tf.math.reduce_sum(y_true, axis=-1) + K.epsilon()
    prediction_value = tf.math.reduce_sum(y_pred, axis=-1) + K.epsilon()

    info_avg = 2 / ((1 / label_value) + (1 / prediction_value))
    diff = multiclass_squared_error / info_avg

    # Root Mean of the normalised squared error
    return K.sqrt(K.mean(diff))


class AttentionWeights(Layer):
    def __init__(
        self,
        step_dim,
        W_regularizer=None,
        b_regularizer=None,
        W_constraint=None,
        b_constraint=None,
        bias=True,
        **kwargs
    ):
        self.supports_masking = True
        self.init = initializers.get("glorot_uniform")
        # self.init = initializers.get(Constant(value=1))

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(AttentionWeights, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(
            shape=(input_shape[-1],),
            initializer=self.init,
            name="{}_W".format(self.name),
            regularizer=self.W_regularizer,
            constraint=self.W_constraint,
        )
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(
                shape=(input_shape[1],),
                initializer="zero",
                name="{}_b".format(self.name),
                regularizer=self.b_regularizer,
                constraint=self.b_constraint,
            )
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(
            K.dot(
                K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))
            ),
            (-1, step_dim),
        )

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        return a

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.features_dim

    def get_config(self):
        config = {"step_dim": self.step_dim}
        base_config = super(AttentionWeights, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


class ContextVector(Layer):
    def __init__(self, **kwargs):
        super(ContextVector, self).__init__(**kwargs)
        self.features_dim = 0

    def build(self, input_shape):
        assert len(input_shape) == 2
        self.features_dim = input_shape[0][-1]
        self.built = True

    def call(self, x, **kwargs):
        assert len(x) == 2
        h = x[0]
        a = x[1]
        a = K.expand_dims(a)
        weighted_input = h * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0][0], self.features_dim

    def get_config(self):
        base_config = super(ContextVector, self).get_config()
        return dict(list(base_config.items()))

## Constants

In [3]:
class C:
    """
    Constants class.
    """

    OBSERVATIONS = 0
    CORRECT_GOAL = 1
    POSSIBLE_GOALS = 2

    SATELLITE = 0
    LOGISTICS = 1
    ZENOTRAVEL = 2
    BLOCKSWORLD = 3
    DRIVERLOG = 4
    DEPOTS = 5

    MAX_PLAN_LENGTH = 0
    MODEL_FILE = 1
    DICTIONARIES_DICT = 2

    SMALL = 0
    COMPLETE = 1
    PERCENTAGE = 2

    CUSTOM_OBJECTS = {
        "AttentionWeights": AttentionWeights,
        "ContextVector": ContextVector,
        "custom_multilabel_loss_v3": custom_multilabel_loss_v3,
    }

    MODELS_DIR = "./models/"
    DICTIONARIES_DIR = "./dictionaries/"
    # MODELS_DIR = './incremental_models/'

    MODEL_LOGISTICS = None
    MODEL_SATELLITE = None
    MODEL_ZENOTRAVEL = None
    MODEL_BLOCKSWORLS = None
    MODEL_DRIVERLOG = None
    MODEL_DEPOTS = None

    MAX_PLAN_PERCENTAGE = 0.7

    TABLE_HEADERS = ["", "Pereira", "Our", "Support"]

## Utils Functions

In [ ]:
def unzip_file(file_path: str, target_dir: str) -> None:
    if os.path.exists(target_dir):
        for f in os.listdir(target_dir):
            os.remove(join(target_dir, f))
        os.rmdir(target_dir)
    os.mkdir(target_dir)
    os.system(f"unzip -qq {file_path} -d {target_dir}")

In [ ]:
def unpack_bz2(file_path: str, target_dir: str) -> None:
    if os.path.exists(target_dir):
        for f in os.listdir(target_dir):
            os.remove(join(target_dir, f))
        os.rmdir(target_dir)
    os.mkdir(target_dir)
    os.system(f"tar -xf {file_path} -C {target_dir}")

In [ ]:
def fix_goals_dictionary(goals_dict_dir: str) -> None:
    goals_dict = load_file(goals_dict_dir, binary=True, use_pickle=True)
    new_dict = dict()
    for k in goals_dict:
        new_dict[k.upper()] = np.argmax(goals_dict[k])
    with open(goals_dict_dir, "wb") as wf:
        pickle.dump(new_dict, wf)
        wf.close()

## GRNet functions

In [4]:
class PlanLengthError(Exception):
    pass


def load_file(file: str, binary: bool = False, use_pickle: bool = False):
    """
    Get file content from path.

    Args:
        file:
            A string that contains the path
            to the file.

        binary:
            Optional. True if the file is a
            binary file.

        use_pickle:
            Optional. True if the file was
            saved using pickle.

    Returns:
        The content of the file.

    Raises:
        FileNotFoundError:
            An error accessing the file
    """
    operation = "r"
    if binary:
        operation += "b"
    with open(file, operation) as rf:
        if use_pickle:
            output = pickle.load(rf)
        else:
            output = rf.readlines()
        rf.close()
    return output

In [5]:
class FileFormatError(Exception):
    pass


class UnknownIndexError(Exception):
    pass


def parse_file(read_file: str, content_type: int, dictionary: dict = None):
    """
    Parse different input files.

    Args:
        read_file:
            String containing the path to the file.
        content_type:
            Integer representing the kind of parse to apply.
                0: observations file,
                1: correct goal file,
                2: possible goals file

    Returns:
        A list of strings that contains the parsed elements.

    Raises:
        FileFormatError:
            An error regarding the action format in
            the file
    """

    msg_empty = f"File {read_file} is empty."
    msg_index = f"Content type {content_type} is unknown."

    elements = list()

    lines = load_file(read_file)
    if len(lines) == 0:
        raise FileFormatError(msg_empty)
    if content_type == C.OBSERVATIONS:
        elements = parse_observations(lines, dictionary)
    elif content_type == C.POSSIBLE_GOALS:
        elements = parse_possible_goals(lines, dictionary)
    elif content_type == C.CORRECT_GOAL:
        elements = parse_correct_goal(lines[0], dictionary)
    else:
        raise UnknownIndexError(msg_index)

    if len(elements) > 0:
        return elements
    else:
        raise FileFormatError(msg_empty)


def remove_parentheses(line: str) -> str:
    """
    Remove parentheses from a string.

    Args:
        line: a string that is enclosed in parentheses.
        For example:

        "(string example)"

    Returns:
        The string without the parenteses.
        None if the string is empty.

    Raises:
        FileFormatError: error handling the string
    """

    msg = (
        f'Error while parsing a line. Expected "(custom ' + f'text)" but found "{line}"'
    )

    line = line.strip()
    if line.startswith("(") and line.endswith(")"):
        element = line[1:-1]
        element = element.strip()
        if len(element) == 0:
            return None
        else:
            return element
    elif len(line) == 0:
        return None
    else:
        raise FileFormatError(msg)


def retrieve_from_dict(key: str, dictionary: dict):
    """
    Return the dictionary value given the key.

    Args:
        key:
            A string that is the key.
        dictionary:
            A dict.

    Returns:
        The value corresponding to the key.

    Raises:
        KeyError:
            An error accessing the dictionary.
    """

    msg_error = f"Key {key.upper()} is not in the dictionary"

    try:
        return dictionary[key.upper()]
    except KeyError:
        print(msg_error)
        np.random.seed(47)
        return np.random.randint(0, len(dictionary))


def parse_correct_goal(line: str, goals_dict: dict = None) -> list:
    """
    Parse the fluents that compose a goal.

    Args:
        line:
            A string that contains one or more
            fluents in the goal. Fluents are
            enclosed in parentheses and separated
            by commas. For example:

            "(fluent1), (fluent2),  (fluent3)"

        goals_dict:
            Optional. A dictionary that maps each
            fluent to its unique identifier.

    Returns:
        A list of strings containing each fluent
        without parentheses.

    Raises:
        FileFormatError:
            An error accessing the file.
    """
    msg_empty = "Parsed goal is empty."

    goal = list()
    line = line.strip()
    fluents = line.split(",")
    for f in fluents:
        fluent = remove_parentheses(f)
        if fluent is not None:
            if goals_dict is not None:
                fluent = retrieve_from_dict(fluent, goals_dict)
            goal.append(fluent)
    if len(goal) > 0:
        return goal
    else:
        raise FileFormatError(msg_empty)


def parse_observations(lines: list, obs_dict: dict = None) -> list:
    """
    Removes parentheses and empty strings from
    the observations list.

    Args:
        lines:
            List of strings that contains the
            observations. Each observation is
            enclosed in parentheses. For
            example:

            ['(observation1)', '', '(observation2)']

        obs_dict:
            Optional. A dictionary that maps each
            observation to its unique identifier.

    Returns:
        The input list without parentheses and
        empty strings.

    Raises:
        FileFormatError:
            An error accessing the file.
    """
    msg_empty = "Observations list is empty."

    observations = list()

    for line in lines:
        observation = remove_parentheses(line)
        if observation is not None:
            if obs_dict is not None:
                observation = retrieve_from_dict(observation, obs_dict)
            observations.append(observation)
    if len(observations) > 0:
        return observations
    else:
        raise FileFormatError(msg_empty)


def parse_possible_goals(lines: list, goals_dict: dict = None) -> list:
    """
    Parse a list of goals.

    Args:
        lines:
            A list of strings that contains each
            possible goal.

        goals_dict:
            Optional. A dictionary that maps each
            fluent to its unique identifier.

    Returns:
        A list of lists. Each list contains the fluents
        that compose the goal represented as a string.

    Raises:
        FileFormatError:
            An error accessing the file.
    """
    msg_empty = "Possible goals list is empty."

    goals = list()
    for line in lines:
        line = line.strip()
        if len(line) > 0:
            goals.append(parse_correct_goal(line, goals_dict))
    if len(goals) > 0:
        return goals
    else:
        raise FileFormatError(msg_empty)

In [6]:
def parse_domain(domain: Union[str, int]) -> int:
    """
    Converts domain name into integer

    Args:
        domain:
            A string or an int that represents
            a domain.

    Returns:
        An integer associated to a specific domain.

    Raises:
        KeyError:
            An error parsing the domain arg.
    """
    msg = (
        f"Provided domain {domain} is not supported. "
        + f"Supported domains are: {C.SATELLITE} : satellite, "
        + f"{C.LOGISTICS} : logistics, {C.BLOCKSWORLD} : blocksworld, "
        + f"{C.ZENOTRAVEL} : zenotravel, {C.DRIVERLOG}: driverlog,"
        + f"{C.DEPOTS}: depots."
    )

    if (str(domain).isdigit() and int(domain) == C.SATELLITE) or str(
        domain
    ).lower().strip() == "satellite":
        return C.SATELLITE
    elif (str(domain).isdigit() and int(domain) == C.LOGISTICS) or str(
        domain
    ).lower().strip() == "logistics":
        return C.LOGISTICS
    elif (str(domain).isdigit() and int(domain) == C.BLOCKSWORLD) or str(
        domain
    ).lower().strip() == "blocksworld":
        return C.BLOCKSWORLD
    elif (str(domain).isdigit() and int(domain) == C.ZENOTRAVEL) or str(
        domain
    ).lower().strip() == "zenotravel":
        return C.ZENOTRAVEL
    elif (str(domain).isdigit() and int(domain) == C.DRIVERLOG) or str(
        domain
    ).lower().strip() == "driverlog":
        return C.DRIVERLOG
    elif (str(domain).isdigit() and int(domain) == C.DEPOTS) or str(
        domain
    ).lower().strip() == "depots":
        return C.DEPOTS
    else:
        raise KeyError(msg)

In [7]:
def get_model(domain: int):
    """
    Loads the model for a specific domain.

    Args:
        domain:
            an integer associated to a specific
            domain.

    Returns:
        The Model loaded for the domain or None
        if there is no model in memory.

    Raises:
        KeyError:
            An error parsing the domain arg.
    """

    msg = (
        f"Provided domain {domain} is not supported. "
        + f"Supported domains are: {C.SATELLITE} : satellite, "
        + f"{C.LOGISTICS} : logistics, {C.BLOCKSWORLD} : blocksworld, "
        + f"{C.ZENOTRAVEL} : zenotravel, {C.DRIVERLOG}: driverlog,"
        + f"{C.DEPOTS}: depots."
    )

    if domain == C.LOGISTICS:
        return C.MODEL_LOGISTICS
    elif domain == C.SATELLITE:
        return C.MODEL_SATELLITE
    elif domain == C.DEPOTS:
        return C.MODEL_DEPOTS
    elif domain == C.BLOCKSWORLD:
        return C.MODEL_BLOCKSWORLS
    elif domain == C.DRIVERLOG:
        return C.MODEL_DRIVERLOG
    elif domain == C.ZENOTRAVEL:
        return C.MODEL_ZENOTRAVEL
    else:
        raise KeyError(msg)

In [108]:
def get_domain_related(
    domain: int, element: int, model_type: int = C.SMALL, percentage: float = 0
) -> Union[int, str]:
    """
    Returns domain related information

    Args:
        domain:
            an integer associated to a specific
            domain.

        element:
            an integer associated to a specific
            piece of information to retrieve.

        model_type:
            an integer associated to the type
            of RNN model in use.

        percentage:
            a float that represents the model
            percentage to use. Use only with
            model_type = C.PERCENTAGE.

    Returns:
        Max plan size if element=C.MAX_PLAN_LENGTH,
        Model file if element=C.MODEL_FILE
        Dictionaries directory if element=C.DICTIONARIES_DICT

    """

    msg = (
        f"Provided domain {domain} is not supported. "
        + f"Supported domains are: {C.SATELLITE} : satellite, "
        + f"{C.LOGISTICS} : logistics, {C.BLOCKSWORLD} : blocksworld, "
        + f"{C.ZENOTRAVEL} : zenotravel."
    )
    if domain == C.LOGISTICS:
        v = {
            "max_plan_len": 50,
            "name": "logistics",
        }
    elif domain == C.SATELLITE:
        v = {
            "max_plan_len": 40,
            "name": "satellite",
        }
    elif domain == C.ZENOTRAVEL:
        v = {
            "max_plan_len": 40,
            "name": "zenotravel",
        }
    elif domain == C.BLOCKSWORLD:
        v = {
            "max_plan_len": 75,
            "name": "blocksworld",
        }
    elif domain == C.DRIVERLOG:
        v = {
            "max_plan_len": 70,
            "name": "driverlog",
        }
    elif domain == C.DEPOTS:
        v = {"max_plan_len": 64, "name": "depots"}
    else:
        raise KeyError(msg)

    if element == C.MAX_PLAN_LENGTH:
        return int(v["max_plan_len"] * C.MAX_PLAN_PERCENTAGE)

    elif element == C.MODEL_FILE:
        if model_type == C.COMPLETE:
            return f'{v["name"]}.h5'
        elif model_type == C.SMALL:
            return f'{v["name"]}_small.h5'
        elif model_type == C.PERCENTAGE:
            return f'{v["name"]}_{int(percentage*100)}perc.h5'

    elif element == C.DICTIONARIES_DICT:
        return join(C.DICTIONARIES_DIR, f'{v["name"]}')


def get_observations_array(observations: list, max_plan_length: int) -> np.ndarray:
    """
    Create an array of observations index.

    Args:
        observations:
            A list of action names

        max_plan_length:
            An integer that contains the maximum size of
            the list that will be considered.

    Returns:
        An array that contains the observations' indexes
    """

    observations_array = np.zeros((1, max_plan_length))
    if len(observations) > max_plan_length:
        print("PLAN TOO LONG")
    for index, observation in enumerate(observations):
        if index < max_plan_length:
            observations_array[0][index] = int(observation)
    return observations_array


def get_predictions(
    observations: list, max_plan_length: int, domain: int
) -> np.ndarray:
    """
    Return the model predictions.

    Args:
        observations:
            A list of action names.

        max_plan_length:
            An integer that contains the maximum size of
            the list that will be considered.

        domain:
            An integer associated to a specific domain.

    Returns:
        The model predictions.
    """

    model = get_model(domain)

    model_input = tf.convert_to_tensor(
        get_observations_array(observations, max_plan_length)
    )
    y_pred = model.predict(model_input)
    return y_pred


def get_score(prediction: np.ndarray, possible_goal: list) -> float:
    """
    Returns the score for a possible goal.

    Args:
        prediction:
            An array that contains the model prediction.

        possible_goal:
            A list that contains the possible goal indexes.

    Returns:
        An float that represents the score of the possible goal.
    """

    score = 0

    for index in possible_goal:
        score += prediction[0][int(index)]
    return score


def get_scores(prediction: np.ndarray, possible_goals: list) -> np.ndarray:
    """
    Returns the scores for all possible goals.

    Args:
        prediction:
            An array that contains the model prediction.

        possible_goals:
            A list of possible goals; each possible goal is represented as a
            list

    Returns:
        An array that contains the score of each of the possible goals.
    """
    scores = np.zeros(
        (
            len(
                possible_goals,
            )
        ),
        dtype=float,
    )
    for index, possible_goal in enumerate(possible_goals):
        scores[index] = get_score(prediction, possible_goal)
    return scores


def get_max(scores: np.ndarray) -> list:
    """
    Returns a list with the index (or indexes) of the highest scores.

    Args:
        scores:
            An array that contains the scores as floats.

    Returns:
        A list thet contains the indexes of the highest score.
    """
    max_element = -1
    index_max = list()
    for i in range(len(scores)):
        if scores[i] > max_element:
            max_element = scores[i]
            index_max = [i]
        elif scores[i] == max_element:
            index_max.append(i)

    return index_max


def get_result(scores: np.ndarray, correct_goal: int) -> bool:
    """
    Computes if the goal recognition task is successfull.

    Args:
        scores:
            An array of floats that contains a score for
            each possible goal

        correct_goal:
            An integer that represents the index of the
            correct goal

    Returns:
        True if the maximum score index corresponds to the
        correct goal index, False otherwise.
    """
    idx_max_list = get_max(scores)
    if len(idx_max_list) == 1:
        idx_max = idx_max_list[0]
    else:
        print(f"Algorithm chose randomly one of {len(idx_max_list)} equals candidates.")
        idx_max = idx_max_list[np.random.randint(0, len(idx_max_list))]
    if idx_max == correct_goal:
        return True
    else:
        return False


def get_correct_goal_idx(correct_goal: list, possible_goals: list) -> int:
    """
    Conputes the correct goal index.

    Args:
        correct_goal:
            A list of strings that contains the correct goal
            fluents.

        possible_goals:
            A list of possible goals; each possible goal is represented as a
            list.

    Returns:
        The index of the correct goal in the possible goals list.
        None if the possible goal list does not contain the correct goal.
    """

    for index, possible_goal in enumerate(possible_goals):
        possible_goal = np.sort(possible_goal)
        correct_goal = np.sort(correct_goal)
        if np.all(possible_goal == correct_goal):
            return index
    return None

In [9]:
def init_models(model_type: int, percentage: float) -> None:
    """
    Loads in memory all the models.

    Args:
        model_type:
            an integer associated to the type
            of RNN model in use.

        percentage:
            a float that represents the model
            percentage to use. Use only with
            model_type = C.PERCENTAGE.

    Returns:
        None
    """

    model_file = get_domain_related(
        C.LOGISTICS, C.MODEL_FILE, model_type=model_type, percentage=percentage
    )
    C.MODEL_LOGISTICS = load_model(
        join(C.MODELS_DIR, model_file), custom_objects=C.CUSTOM_OBJECTS
    )

    model_file = get_domain_related(
        C.SATELLITE, C.MODEL_FILE, model_type=model_type, percentage=percentage
    )
    C.MODEL_SATELLITE = load_model(
        join(C.MODELS_DIR, model_file), custom_objects=C.CUSTOM_OBJECTS
    )

    model_file = get_domain_related(
        C.ZENOTRAVEL, C.MODEL_FILE, model_type=model_type, percentage=percentage
    )
    C.MODEL_ZENOTRAVEL = load_model(
        join(C.MODELS_DIR, model_file), custom_objects=C.CUSTOM_OBJECTS
    )

    model_file = get_domain_related(
        C.DEPOTS, C.MODEL_FILE, model_type=model_type, percentage=percentage
    )
    C.MODEL_DEPOTS = load_model(
        join(C.MODELS_DIR, model_file), custom_objects=C.CUSTOM_OBJECTS
    )

    model_file = get_domain_related(
        C.DRIVERLOG, C.MODEL_FILE, model_type=model_type, percentage=percentage
    )
    C.MODEL_DRIVERLOG = load_model(
        join(C.MODELS_DIR, model_file), custom_objects=C.CUSTOM_OBJECTS
    )

    model_file = get_domain_related(
        C.BLOCKSWORLD, C.MODEL_FILE, model_type=model_type, percentage=percentage
    )
    C.MODEL_BLOCKSWORLS = load_model(
        join(C.MODELS_DIR, model_file), custom_objects=C.CUSTOM_OBJECTS
    )

In [114]:
def run_experiment(
    obs_file: str,
    goals_dict_file: Union[str, None],
    actions_dict_file: Union[str, None],
    possible_goals_file: str,
    correct_goal_file: str,
    domain: Union[str, int],
    verbose: int = 0,
) -> list:
    """
    Run the goal recognition experiment

    Args:
        obs_file:
            Path of the file that contains the
            observations (plan)

        goals_dict_file:
            Path of the file that contains the
            goals dictionaries. If None it is
            retrieved from its default location.

        actions_dict_file:
            Path of the file that contains the
            actions dictionaries. If None it is
            retrieved from its default location.

        possible_goals_file:
            Path of the file that contains the
            possible goals.

        correct_goal_file:
            Path of the file that contains the
            correct goal.

        domain:
            String that contains the name of the
            domain or integer that corresponds to
            a domain.

        verbose:
            Integer that corresponds to how much
            information is printed. 0 = no info,
            2 = max info

    Returns:
         A list that contains the result, the correct
         goal index and the predicted goal index.
    """

    domain = parse_domain(domain)
    if goals_dict_file is None:
        goals_dict_file = join(
            get_domain_related(domain, C.DICTIONARIES_DICT), "dizionario_goal"
        )
    goals_dict = load_file(goals_dict_file, binary=True, use_pickle=True)
    if actions_dict_file is None:
        actions_dict_file = join(
            get_domain_related(domain, C.DICTIONARIES_DICT), "dizionario"
        )
    actions_dict = load_file(actions_dict_file, binary=True, use_pickle=True)
    observations = parse_file(obs_file, C.OBSERVATIONS, actions_dict)

    for k in goals_dict:
        print(k, goals_dict[k])

    if verbose > 1:
        print("Observed actions:\n")
        for o in observations:
            print(o)
    possible_goals = parse_file(possible_goals_file, C.POSSIBLE_GOALS, goals_dict)

    max_plan_length = get_domain_related(domain, C.MAX_PLAN_LENGTH)
    predictions = get_predictions(observations, max_plan_length, domain)
    scores = get_scores(predictions, possible_goals)
    if verbose > 0:
        for index, goal in enumerate(possible_goals):
            print(f"{index} - {goal} : {scores[index]}")

    correct_goal = parse_file(correct_goal_file, C.CORRECT_GOAL, goals_dict)
    correct_goal_idx = get_correct_goal_idx(correct_goal, possible_goals)
    result = get_result(scores, correct_goal_idx)
    if verbose > 0:
        print(f"Predicted goal is {get_max(scores)[0]}")
        print(f"Correct goal is {correct_goal_idx} - {correct_goal}")
    return [result, correct_goal_idx, get_max(scores)[0]]

## Results utils

In [53]:
def get_results_list(buckets_number: int) -> list:
    """
    Creates an empty list to store buckets
    results.

    Args:
        buckets_number:
            The number of buckets

    Returns:
        A list that contains an empty list
        for each bucket
    """
    results = list()
    for _ in range(buckets_number):
        results.append(list())
    return results

In [ ]:
def get_accuracy(output_dir: str) -> float:
    """
    Computes accuracy score for pereira results

    Args:
        output_dir:
            Path to the folder that contains the
            outputs of pereira's GR program

    Returns:
         The accuracy score for the results
    """
    outputs = os.listdir(output_dir)
    results = list()

    for output in outputs:
        if output.startswith("p") or output.startswith("output_"):
            with open(join(output_dir, output), "r") as rf:
                lines = rf.readlines()
                if "true" in lines[-1]:
                    to_append = True
                elif "false" in lines[-1]:
                    to_append = False
                else:
                    to_append = None
                results.append(to_append)
                rf.close()
    return results.count(True) / len(results)


def get_accuracy_bucket(output_dir: str) -> list:
    """
    Computes accuracy score for pereira results
    with buckets.

    Args:
        output_dir:
            Path to the folder that contains the
            outputs of pereira's GR program

    Returns:
         A list that contains the accuracy score
         for each of the bucket
    """
    outputs = os.listdir(output_dir)
    results = list()
    count = np.zeros((9,))
    count_tot = np.zeros((9,))
    to_ret = np.zeros((9,))
    for output in outputs:
        for b in range(9):
            if f"b=0{b+1}" in output:
                if output.startswith("p") or output.startswith("output_"):
                    with open(join(output_dir, output), "r") as rf:
                        lines = rf.readlines()
                        if len(lines) > 0:
                            if "true" in lines[-1]:
                                count[b] += 1
                            count_tot[b] += 1
                            rf.close()
    for i in range(9):
        to_ret[i] = count[i] / count_tot[i]
    return to_ret

In [ ]:
def create_table(
    title: str, headers: list, rows: list, just: int = 10, precision: int = 2
) -> list:
    """
    Prints a table.

    Args:
        title:
            The title of the table

        headers:
            A list that contains the header of
            the table

        rows:
            A list that contains each row
            represented as a list

        just:
            Integer that represent the spacing
            between the columns

        precision:
            Integer that represents the precision
            in representing folat numbers.

    Returns:
         A list that contains the table. Each
         element is a string that contains a
         row.
    """
    to_ret = []
    divider = "-" * ((just + 5) * len(headers) - 4)
    to_ret.append(title)
    to_ret.append(divider)
    s = ""
    for h in headers:
        s += f"{str(h).rjust(just)}     "
    to_ret.append(s)
    to_ret.append(divider)

    for row in rows:
        s = ""
        for value in row:
            if type(value) == int:
                s += f"{value:{just}.{precision}f}     "
            elif type(value) == float or type(value) == np.float64:
                s += f"{value:{just}.{precision}f}     "
            elif type(value) == str:
                s += f"{value.rjust(just)}     "
            else:
                print(f"Could not parse type {type(value)}")
        to_ret.append(s)
    to_ret.append(divider)
    return to_ret

In [ ]:
def print_comparison_results(
    domain: str,
    our_res: list,
    our_times: list,
    p_res: str,
    p_times: list,
    latex: bool = False,
    perc: list = [30, 50, 70],
) -> None:
    """
    Print a results comparison

    Args:
        domain:
            String that represents the domain

        our_res:
            List of lists. Each list contains
            the results for a different
            observability percentage.

        our_times:
            Lists of lists. Each list contains
            the times for a different
            observability percentage.

        p_res:
            Folder that contains the outputs of
            pereira's GR program.

        p_times:
            Lists of lists. Each list contains
            the times for a different
            observability percentage.

        latex:
            If True prints the results as latex
            table

        perc:
            List that contains the percentages
            as int

    Returns:
         None
    """

    print(f"{domain.upper()}\n")

    acc = np.zeros((len(perc), len(C.TABLE_HEADERS)))
    for i, perc_results in enumerate(our_res):
        acc[i][0] = perc[i]
        acc[i][2] = accuracy_score(perc_results[0], perc_results[1]) * 100
    for i, p in enumerate(perc):
        acc[i][1] = get_accuracy(f"./{p_res}/{p}/") * 100
        if len(perc_results[0]) == len(perc_results[1]):
            acc[i][3] = len(perc_results[0])
        else:
            acc[i][3] = None

    table = create_table(
        title="Accuracy comparison", headers=C.TABLE_HEADERS, rows=acc, just=20
    )

    for t in table:
        print(t)

    if latex:
        s = ""
        for row in acc:
            for el in row[1:-1]:
                if s != "":
                    s += " & "
                s += f"{el:.1f}"
        print("\nLatex accuracy")
        print(s)

    print()

    times = np.zeros((len(perc) + 1, len(C.TABLE_HEADERS)))
    if our_times is None or p_times is None or len(our_times) != len(p_times):
        return

    third = int(len(our_times) / 3)
    for i, p in enumerate(perc):
        times[i][0] = perc[i]
        times[i][1] = np.mean(p_times[i * third : (i + 1) * third]) * 1000
        times[i][2] = np.mean(our_times[i * third : (i + 1) * third]) * 1000
        times[i][3] = third
    times[len(perc)][0] = None
    times[len(perc)][1] = np.mean(p_times) * 1000
    times[len(perc)][2] = np.mean(our_times) * 1000
    times[len(perc)][3] = len(our_times)

    table = create_table(
        title="Times comparison",
        headers=C.TABLE_HEADERS,
        rows=times,
        just=20,
        precision=0,
    )

    for t in table:
        print(t)

    if latex:
        s = ""
        for row in times:
            for el in row[1:-1]:
                if s != "":
                    s += " & "
                s += f"{el:.0f}"
        print("\nLatex times")
        print(s)

In [ ]:
def split_goal(goal: str) -> list:
    """
    Split the goal in its fluents

    Args:
        goal:
            The string that contains the
            goal represented as:

            "(and (fluent1) ... (fluentN))"

    Returns:
        A list that contains each fluent as
        a string.
    """
    goal = goal.split("and", 1)[1]
    goal = goal.rsplit(")", 1)[0]
    goal = goal.split("(")
    for i, _ in enumerate(goal):
        goal[i] = goal[i].strip()
    return goal[1:]


def check_goals(candidate: list, goals: list) -> Union[int, None]:
    """
    Check if the candidate is in the goals

    Args:
        candidate:
            A list that contains the fluents of
            the candidate

        goals:
            A list that contains the possible
            goals

    Returns:
         The index of the candidate in the goals
         list or None if the candidate is not
         included in the list.
    """
    for i, goal in enumerate(goals):
        goal_found = True
        if len(candidate) == len(goal):
            for fact in candidate:
                if fact not in goal:
                    goal_found = False
                    break
        else:
            goal_found = False
        if goal_found:
            return i

    return None


def get_class_report(output_dir: str) -> list:
    """
    Returns the classification report for
    pereira's GR program.

    Args:
        output_dir:
            Path to the folder that contains the
            outputs of pereira's GR program

    Returns:
         The classification report as a list of
         stirngs.
    """
    outputs = os.listdir(output_dir)
    goals = list()
    y_true = list()
    y_pred = list()

    for j, output in enumerate(outputs):
        if output.startswith("p"):

            correct_index = -1
            recognised_index = -1
            skip_recognised = False

            with open(join(output_dir, output), "r") as rf:
                lines = rf.readlines()
                rf.close()

            if "true" in lines[-1]:
                skip_recognised = True

            for i in range(len(lines) - 1, -1, -1):
                l = lines[i].strip()
                if not skip_recognised and l.startswith("$> Recognized goal(s)"):
                    recognised_index = i + 1
                elif l.startswith("<?> Correct goal: "):
                    correct_index = i
                if correct_index != -1 and (skip_recognised or recognised_index != -1):
                    break

            if correct_index != -1:
                correct_goal = lines[correct_index].split("Correct goal:", 1)
                correct_goal = correct_goal[1].strip()
                correct_goal = split_goal(correct_goal)
                res = check_goals(correct_goal, goals)
                if res is None:
                    goals.append(set(correct_goal))
                    res = len(goals) - 1
                y_true.append(res)

                if skip_recognised:
                    y_pred.append(res)

                elif recognised_index != -1:
                    rec_goal = lines[recognised_index].split("$> ", 1)[1]
                    rec_goal = rec_goal.split(":", 1)[0]
                    rec_goal = split_goal(rec_goal)
                    res = check_goals(rec_goal, goals)
                    if res is None:
                        goals.append(set(rec_goal))
                        res = len(goals) - 1
                    y_pred.append(res)
            else:
                v = random.randint(1, 4)
                y_pred.append(v)
                y_true.append(v + 1)
    labels = list()
    for goal in goals:
        labels.append(str(goal))
    return classification_report(y_true, y_pred, target_names=labels)

## GRNet executions

In [48]:
model_type = C.SMALL
percentage = 0
init_models(model_type=model_type, percentage=percentage)

perc_list = [0.3, 0.5, 0.7]
source_dir = "./test_instance_pereira/"
results = list()
times = list()
for perc in perc_list:
    plans_dir = f"../goal-plan-recognition-dataset/zeno-travel/{int(perc*100)}"
    files = os.listdir(plans_dir)
    total = 0
    correct = 0
    results_file = [list(), list()]
    for j, f in enumerate(files):
        print(f)
        if f.endswith(".zip"):
            unzip_file(join(plans_dir, f), source_dir)
        elif f.endswith(".bz2"):
            unpack_bz2(join(plans_dir, f), source_dir)
        start_time = time.time()
        result = run_experiment(
            obs_file=join(source_dir, "obs.dat"),
            goals_dict_file=None,
            actions_dict_file=None,
            possible_goals_file=join(source_dir, "hyps.dat"),
            correct_goal_file=join(source_dir, "real_hyp.dat"),
            domain=C.ZENOTRAVEL,
            verbose=1,
        )
        exec_time = time.time() - start_time
        if result[0]:
            correct += 1
        total += 1
        times.append(exec_time)
        print(exec_time)
        results_file[0].append(result[1])
        results_file[1].append(result[2])
    results.append(results_file)

zeno-travel_p03_hyp-4_30_1.tar.bz2
0 - [5, 52, 35, 26, 30, 2] : 0.4580400586128235
1 - [50, 52, 35, 14, 45, 2] : 1.4054368138313293
2 - [6, 16, 46, 4, 37, 62] : 0.31425678730010986
3 - [50, 12, 33, 14, 30, 11] : 1.5267949104309082
4 - [5, 16, 46, 14, 30, 62] : 1.3662010729312897
5 - [6, 12, 33, 14, 37, 62] : 1.4122051000595093
Predicted goal is 3
Correct goal is 3 - [50, 12, 33, 14, 30, 11]
0.3638155460357666
zeno-travel_p06_hyp-1_30_1.tar.bz2
0 - [56, 12, 46, 14, 8, 2, 43] : 3.521370977163315
1 - [56, 12, 35, 27, 45, 62, 7] : 0.34287071228027344
2 - [6, 52, 3, 26, 37, 25, 44] : 0.2324085235595703
3 - [6, 52, 18, 49, 45, 25, 47] : 0.24204370379447937
4 - [5, 20, 18, 14, 37, 25, 15] : 1.3917056024074554
5 - [34, 16, 33, 14, 30, 25, 7] : 1.4292588531970978
Predicted goal is 0
Correct goal is 0 - [56, 12, 46, 14, 8, 2, 43]
0.09542369842529297
zeno-travel_p05_hyp-3_30_1.tar.bz2
0 - [16, 46, 26, 8, 11, 55, 29, 24] : 0.8125295042991638
1 - [16, 46, 27, 45, 25, 47, 22, 59] : 0.723493158817291

0 - [5, 52, 35, 26, 30, 2] : 0.5742630958557129
1 - [50, 52, 35, 14, 45, 2] : 1.4651863873004913
2 - [6, 16, 46, 4, 37, 62] : 0.18192383646965027
3 - [50, 12, 33, 14, 30, 11] : 2.3781875371932983
4 - [5, 16, 46, 14, 30, 62] : 1.2760803699493408
5 - [6, 12, 33, 14, 37, 62] : 1.2959414422512054
Predicted goal is 3
Correct goal is 3 - [50, 12, 33, 14, 30, 11]
0.0809638500213623
zeno-travel_p07_hyp-2_30_1.tar.bz2
0 - [54, 52, 33, 27, 37, 0, 15, 51, 36] : 0.9577169716358185
1 - [54, 52, 3, 26, 53, 60, 7, 61, 17] : 1.7441612482070923
2 - [28, 65, 18, 14, 53, 11, 47, 51, 23] : 2.5222229063510895
3 - [5, 65, 40, 4, 45, 62, 44, 13, 17] : 0.4538753032684326
4 - [50, 48, 35, 26, 53, 25, 55, 29, 59] : 1.800707072019577
5 - [39, 12, 35, 14, 8, 2, 47, 22, 24] : 1.8561831414699554
Predicted goal is 2
Correct goal is 2 - [28, 65, 18, 14, 53, 11, 47, 51, 23]
0.09501814842224121
zeno-travel_p04_hyp-4_30_3.tar.bz2
0 - [50, 48, 46, 4, 37, 0, 44, 61] : 0.6947021484375
1 - [63, 20, 35, 4, 37, 25, 55, 29] : 

0 - [12, 33, 14, 45, 43] : 0.5072883069515228
1 - [12, 46, 4, 53, 62] : 0.850527822971344
2 - [52, 35, 26, 53, 60] : 0.6302279531955719
3 - [47, 35, 26, 37, 25] : 0.4851769208908081
4 - [16, 3, 49, 30, 2, 43] : 0.9872007369995117
5 - [48, 35, 14, 37, 60, 15] : 0.9146196246147156
Predicted goal is 4
Correct goal is 1 - [12, 46, 4, 53, 62]
0.08736491203308105
zeno-travel_p05_hyp-2_30_3.tar.bz2
0 - [16, 46, 26, 8, 11, 55, 29, 24] : 0.2654748857021332
1 - [16, 46, 27, 45, 25, 47, 22, 59] : 0.19913694262504578
2 - [20, 33, 4, 41, 60, 43, 58, 36] : 4.409517198801041
3 - [20, 33, 14, 8, 2, 55, 51, 17] : 1.6229925453662872
4 - [65, 3, 27, 30, 0, 15, 51, 23] : 0.4019967317581177
5 - [65, 35, 14, 45, 25, 43, 13, 57] : 0.18815168738365173
Predicted goal is 2
Correct goal is 2 - [20, 33, 4, 41, 60, 43, 58, 36]
0.09541130065917969
zeno-travel_p03_hyp-1_30_3.tar.bz2
0 - [5, 52, 35, 26, 30, 2] : 1.7012117803096771
1 - [50, 52, 35, 14, 45, 2] : 0.7731155157089233
2 - [6, 16, 46, 4, 37, 62] : 0.4383866

0 - [56, 12, 46, 14, 8, 2, 43] : 0.4473305344581604
1 - [56, 12, 35, 27, 45, 62, 7] : 0.5370066463947296
2 - [6, 52, 3, 26, 37, 25, 44] : 0.4213946461677551
3 - [6, 52, 18, 49, 45, 25, 47] : 0.5036415755748749
4 - [5, 20, 18, 14, 37, 25, 15] : 0.37561291456222534
5 - [34, 16, 33, 14, 30, 25, 7] : 1.4281402230262756
Predicted goal is 5
Correct goal is 5 - [34, 16, 33, 14, 30, 25, 7]
0.08529114723205566
zeno-travel_p03_hyp-1_30_2.tar.bz2
0 - [5, 52, 35, 26, 30, 2] : 1.1421536803245544
1 - [50, 52, 35, 14, 45, 2] : 1.041472852230072
2 - [6, 16, 46, 4, 37, 62] : 1.5264518558979034
3 - [50, 12, 33, 14, 30, 11] : 1.0946183800697327
4 - [5, 16, 46, 14, 30, 62] : 0.8570807576179504
5 - [6, 12, 33, 14, 37, 62] : 1.4080810248851776
Predicted goal is 2
Correct goal is 0 - [5, 52, 35, 26, 30, 2]
0.09889388084411621
zeno-travel_p04_hyp-3_30_2.tar.bz2
0 - [50, 48, 46, 4, 37, 0, 44, 61] : 0.6561781167984009
1 - [63, 20, 35, 4, 37, 25, 55, 29] : 0.6420561075210571
2 - [63, 52, 35, 49, 8, 0, 7, 51] : 1

0 - [50, 48, 46, 4, 37, 0, 44, 61] : 0.9483397901058197
1 - [63, 20, 35, 4, 37, 25, 55, 29] : 1.5832065343856812
2 - [63, 52, 35, 49, 8, 0, 7, 51] : 1.1240546107292175
3 - [1, 16, 40, 26, 30, 2, 15, 13] : 0.7382095456123352
4 - [50, 16, 46, 49, 37, 25, 47, 61] : 1.1402065753936768
5 - [50, 20, 33, 14, 37, 2, 43, 58] : 0.6060827374458313
6 - [63, 16, 46, 4, 37, 11, 47, 29] : 1.336640328168869
7 - [1, 16, 3, 27, 41, 2, 47, 61] : 0.7105773091316223
8 - [1, 12, 46, 21, 53, 60, 47, 13] : 0.6877874433994293
9 - [1, 16, 18, 21, 53, 2, 15, 29] : 0.9084741771221161
Predicted goal is 1
Correct goal is 1 - [63, 20, 35, 4, 37, 25, 55, 29]
0.10065531730651855
zeno-travel_p07_hyp-2_30_3.tar.bz2
0 - [54, 52, 33, 27, 37, 0, 15, 51, 36] : 0.547092854976654
1 - [54, 52, 3, 26, 53, 60, 7, 61, 17] : 1.4624951481819153
2 - [28, 65, 18, 14, 53, 11, 47, 51, 23] : 2.7158161401748657
3 - [5, 65, 40, 4, 45, 62, 44, 13, 17] : 0.6782637536525726
4 - [50, 48, 35, 26, 53, 25, 55, 29, 59] : 1.5529395937919617
5 - [3

...

0 - [54, 52, 33, 27, 37, 0, 15, 51, 36] : 0.6743356585502625
1 - [54, 52, 3, 26, 53, 60, 7, 61, 17] : 1.5650684535503387
2 - [28, 65, 18, 14, 53, 11, 47, 51, 23] : 1.387510746717453
3 - [5, 65, 40, 4, 45, 62, 44, 13, 17] : 0.36769869923591614
4 - [50, 48, 35, 26, 53, 25, 55, 29, 59] : 4.347131162881851
5 - [39, 12, 35, 14, 8, 2, 47, 22, 24] : 1.4760253429412842
Predicted goal is 4
Correct goal is 4 - [50, 48, 35, 26, 53, 25, 55, 29, 59]
0.09833168983459473
zeno-travel_p05_hyp-2_50_3.tar.bz2
0 - [16, 46, 26, 8, 11, 55, 29, 24] : 0.2774186134338379
1 - [16, 46, 27, 45, 25, 47, 22, 59] : 0.5689320862293243
2 - [20, 33, 4, 41, 60, 43, 58, 36] : 4.632677435874939
3 - [20, 33, 14, 8, 2, 55, 51, 17] : 1.4727689027786255
4 - [65, 3, 27, 30, 0, 15, 51, 23] : 0.7756464779376984
5 - [65, 35, 14, 45, 25, 43, 13, 57] : 1.1430432498455048
Predicted goal is 2
Correct goal is 2 - [20, 33, 4, 41, 60, 43, 58, 36]
0.0980682373046875
zeno-travel_p02_hyp-1_50_1.tar.bz2
0 - [12, 33, 14, 45, 43] : 3.09208661

0 - [54, 52, 33, 27, 37, 0, 15, 51, 36] : 0.8751058876514435
1 - [54, 52, 3, 26, 53, 60, 7, 61, 17] : 1.2403483986854553
2 - [28, 65, 18, 14, 53, 11, 47, 51, 23] : 5.739049166440964
3 - [5, 65, 40, 4, 45, 62, 44, 13, 17] : 0.20259591937065125
4 - [50, 48, 35, 26, 53, 25, 55, 29, 59] : 1.3304796516895294
5 - [39, 12, 35, 14, 8, 2, 47, 22, 24] : 1.945911169052124
Predicted goal is 2
Correct goal is 2 - [28, 65, 18, 14, 53, 11, 47, 51, 23]
0.09571671485900879
zeno-travel_p05_hyp-1_70_3.tar.bz2
0 - [16, 46, 26, 8, 11, 55, 29, 24] : 4.342427849769592
1 - [16, 46, 27, 45, 25, 47, 22, 59] : 0.7059521079063416
2 - [20, 33, 4, 41, 60, 43, 58, 36] : 0.2885381877422333
3 - [20, 33, 14, 8, 2, 55, 51, 17] : 1.3564693331718445
4 - [65, 3, 27, 30, 0, 15, 51, 23] : 0.43232637643814087
5 - [65, 35, 14, 45, 25, 43, 13, 57] : 0.2202342450618744
Predicted goal is 0
Correct goal is 0 - [16, 46, 26, 8, 11, 55, 29, 24]
0.09679579734802246
zeno-travel_p04_hyp-2_70_1.tar.bz2
0 - [50, 48, 46, 4, 37, 0, 44, 61] 

0 - [12, 33, 14, 45, 43] : 0.2969542145729065
1 - [12, 46, 4, 53, 62] : 0.8507710099220276
2 - [52, 35, 26, 53, 60] : 1.2781937420368195
3 - [47, 35, 26, 37, 25] : 2.2826491594314575
4 - [16, 3, 49, 30, 2, 43] : 0.3234763741493225
5 - [48, 35, 14, 37, 60, 15] : 1.6971940696239471
Predicted goal is 3
Correct goal is 3 - [47, 35, 26, 37, 25]
0.11125802993774414
zeno-travel_p02_hyp-3_70_1.tar.bz2
0 - [12, 33, 14, 45, 43] : 0.19430279731750488
1 - [12, 46, 4, 53, 62] : 1.5680446922779083
2 - [52, 35, 26, 53, 60] : 2.6795403361320496
3 - [47, 35, 26, 37, 25] : 0.7894484996795654
4 - [16, 3, 49, 30, 2, 43] : 0.2935941815376282
5 - [48, 35, 14, 37, 60, 15] : 1.5225172638893127
Predicted goal is 2
Correct goal is 2 - [52, 35, 26, 53, 60]
0.09613609313964844
zeno-travel_p01_hyp-1_70_1.tar.bz2
0 - [20, 46, 49, 45, 62] : 4.069125682115555
1 - [16, 46, 4, 30, 25] : 1.158090889453888
2 - [52, 35, 49, 37, 25] : 0.11507046222686768
3 - [16, 3, 14, 37, 2] : 0.043269962072372437
4 - [52, 3, 4, 53, 62] 

0 - [16, 46, 26, 8, 11, 55, 29, 24] : 4.030072569847107
1 - [16, 46, 27, 45, 25, 47, 22, 59] : 1.2474218010902405
2 - [20, 33, 4, 41, 60, 43, 58, 36] : 0.335387647151947
3 - [20, 33, 14, 8, 2, 55, 51, 17] : 1.5330423712730408
4 - [65, 3, 27, 30, 0, 15, 51, 23] : 0.7739339768886566
5 - [65, 35, 14, 45, 25, 43, 13, 57] : 0.4023005962371826
Predicted goal is 0
Correct goal is 0 - [16, 46, 26, 8, 11, 55, 29, 24]
0.10425448417663574
zeno-travel_p06_hyp-1_70_3.tar.bz2
0 - [56, 12, 46, 14, 8, 2, 43] : 4.438959121704102
1 - [56, 12, 35, 27, 45, 62, 7] : 0.3079220950603485
2 - [6, 52, 3, 26, 37, 25, 44] : 0.21692964434623718
3 - [6, 52, 18, 49, 45, 25, 47] : 0.19735053181648254
4 - [5, 20, 18, 14, 37, 25, 15] : 1.2260690331459045
5 - [34, 16, 33, 14, 30, 25, 7] : 1.3449904024600983
Predicted goal is 0
Correct goal is 0 - [56, 12, 46, 14, 8, 2, 43]
0.0963904857635498
zeno-travel_p02_hyp-1_70_3.tar.bz2
0 - [12, 33, 14, 45, 43] : 2.0459904074668884
1 - [12, 46, 4, 53, 62] : 0.41609126329421997
2 -

0 - [20, 46, 49, 45, 62] : 0.418651819229126
1 - [16, 46, 4, 30, 25] : 0.47916093468666077
2 - [52, 35, 49, 37, 25] : 3.1227478981018066
3 - [16, 3, 14, 37, 2] : 1.2410917282104492
4 - [52, 3, 4, 53, 62] : 1.1463248133659363
5 - [20, 35, 14, 30, 62] : 1.3079713881015778
6 - [12, 46, 26, 45, 25] : 0.4815840423107147
7 - [16, 33, 4, 53, 11] : 0.2770356237888336
Predicted goal is 2
Correct goal is 2 - [52, 35, 49, 37, 25]
0.10371661186218262
zeno-travel_p07_hyp-4_70_3.tar.bz2
PLAN TOO LONG
0 - [54, 52, 33, 27, 37, 0, 15, 51, 36] : 0.7674677073955536
1 - [54, 52, 3, 26, 53, 60, 7, 61, 17] : 0.6187354028224945
2 - [28, 65, 18, 14, 53, 11, 47, 51, 23] : 1.591370016336441
3 - [5, 65, 40, 4, 45, 62, 44, 13, 17] : 0.5524674952030182
4 - [50, 48, 35, 26, 53, 25, 55, 29, 59] : 0.45044174790382385
5 - [39, 12, 35, 14, 8, 2, 47, 22, 24] : 3.8030595779418945
Predicted goal is 5
Correct goal is 5 - [39, 12, 35, 14, 8, 2, 47, 22, 24]
0.1197209358215332
zeno-travel_p03_hyp-1_70_3.tar.bz2
0 - [5, 52, 35

### Results

In [49]:
print(results)
print(times)

[[[3, 0, 4, 4, 2, 0, 2, 3, 4, 0, 2, 0, 5, 5, 1, 1, 1, 0, 3, 2, 3, 5, 2, 0, 0, 3, 3, 0, 3, 2, 3, 2, 1, 3, 5, 1, 2, 0, 0, 3, 0, 3, 0, 4, 1, 0, 2, 4, 2, 2, 4, 5, 0, 0, 1, 5, 0, 2, 2, 5, 3, 4, 2, 3, 0, 2, 0, 2, 1, 1, 4, 1, 5, 1, 2, 0, 3, 1, 3, 0, 2, 5, 0, 4], [3, 0, 4, 0, 2, 0, 2, 5, 4, 5, 6, 0, 3, 5, 7, 6, 5, 0, 3, 2, 9, 1, 2, 0, 0, 3, 3, 4, 3, 3, 6, 2, 1, 3, 5, 4, 2, 0, 0, 6, 5, 3, 0, 4, 1, 0, 2, 4, 2, 2, 4, 5, 0, 1, 0, 5, 2, 2, 2, 1, 3, 4, 2, 5, 8, 2, 0, 2, 1, 4, 5, 1, 5, 1, 2, 2, 5, 0, 3, 0, 2, 5, 0, 4]], [[3, 1, 2, 2, 2, 5, 0, 3, 4, 3, 2, 0, 2, 0, 2, 3, 0, 3, 2, 5, 1, 5, 3, 3, 1, 4, 3, 3, 2, 1, 2, 5, 2, 0, 0, 1, 4, 0, 0, 2, 0, 1, 4, 0, 0, 4, 0, 0, 0, 2, 3, 2, 5, 5, 1, 3, 0, 2, 5, 3, 1, 1, 2, 1, 1, 1, 4, 0, 3, 0, 0, 3, 0, 3, 4, 0, 5, 4, 2, 0, 4, 5, 2, 2], [5, 1, 2, 2, 2, 5, 4, 3, 4, 3, 2, 0, 2, 0, 2, 6, 0, 3, 2, 5, 1, 5, 3, 3, 6, 4, 3, 3, 2, 1, 3, 5, 2, 0, 0, 0, 4, 0, 0, 2, 4, 1, 4, 0, 4, 2, 0, 0, 0, 2, 3, 2, 5, 5, 1, 3, 0, 2, 5, 3, 1, 1, 2, 1, 1, 1, 5, 0, 3, 0, 0, 1, 1, 3, 4, 0, 5, 4,

In [108]:
for perc_results in results:
    print(accuracy_score(perc_results[0], perc_results[1]))
print(np.mean(times))

0.6666666666666666
0.8333333333333334
0.8452380952380952
0.1190202065876552


## Pereira's GR program execution

In [ ]:
# perc_list = [0.25, 0.5, 0.75]
perc_list = [0.3, 0.5, 0.7]
source_dir = "/data/users/mchiari/goal_recognition/notebook/test_instance_pereira/"
target_dir = "/data/users/mchiari/goal_recognition/notebook/output_zeno_probatory_1/"
os.makedirs(target_dir, exist_ok=True)
pereira_times = list()
for perc in perc_list:
    plans_dir = (
        f"/data/users/mchiari/goal_recognition/tasks_probatory_1/{int(perc*100)}"
    )
    exp_dir = join(target_dir, f"{int(perc*100)}")
    os.makedirs(exp_dir, exist_ok=True)
    files = os.listdir(plans_dir)
    for i, f in enumerate(files):
        target_file = join(exp_dir, f"output_{f}.txt")
        unzip_file(join(plans_dir, f), source_dir)
        command1 = f"cd {source_dir}"
        command2 = (
            f"java -jar /data/users/mchiari/goal_recognition/Landmark-Based-GoalRecognition/goalrecognizer1.2.jar -uniqueness domain.pddl"
            + f" template.pddl hyps.dat obs.dat real_hyp.dat 0 > {target_file}"
        )
        print(command1)
        start_time = time.time()
        os.system(f"{command1} && {command2}")
        exec_time = time.time() - start_time
        pereira_times.append(exec_time)
        print(exec_time)

cd /data/users/mchiari/goal_recognition/notebook/test_instance_pereira/
3.0777416229248047
cd /data/users/mchiari/goal_recognition/notebook/test_instance_pereira/
0.24766898155212402
cd /data/users/mchiari/goal_recognition/notebook/test_instance_pereira/
0.1733696460723877
cd /data/users/mchiari/goal_recognition/notebook/test_instance_pereira/
0.21805930137634277
cd /data/users/mchiari/goal_recognition/notebook/test_instance_pereira/
0.1613483428955078
cd /data/users/mchiari/goal_recognition/notebook/test_instance_pereira/
0.17018914222717285
cd /data/users/mchiari/goal_recognition/notebook/test_instance_pereira/
0.226546049118042
cd /data/users/mchiari/goal_recognition/notebook/test_instance_pereira/
0.9465210437774658
cd /data/users/mchiari/goal_recognition/notebook/test_instance_pereira/
0.18616962432861328
cd /data/users/mchiari/goal_recognition/notebook/test_instance_pereira/
0.16315364837646484
cd /data/users/mchiari/goal_recognition/notebook/test_instance_pereira/
0.186156511306

### Results

In [18]:
l = list()
print("[")
for p in [30, 50, 70]:
    l = get_accuracy_bucket(f"./output_zeno_probatory_1/{p}")
    l = np.dot(l, 100)
    a = list(l)
    print(f"{list(a)},")
print("]")


print(l)

[
[40.0, 37.0, 37.37373737373738, 40.20618556701031, 38.0, 33.33333333333333, 31.63265306122449, 44.329896907216494, 41.0],
[50.0, 56.99999999999999, 57.99999999999999, 57.99999999999999, 61.0, 59.0, 72.0, 60.0, 68.0],
[79.0, 82.0, 81.0, 86.0, 86.0, 87.0, 85.0, 91.0, 84.0],
]
[79. 82. 81. 86. 86. 87. 85. 91. 84.]


## Results comparisons

## Percentage GRNet experiment

In [299]:
perc_list = [0.3, 0.5, 0.7]
source_dir = "./test_instance_pereira/"
results = list()
times = list()
for perc in perc_list:
    plans_dir = f"../goal-plan-recognition-dataset/satellite//{int(perc*100)}"
    files = os.listdir(plans_dir)
    total = 0
    correct = 0
    results_file = [list(), list()]
    for j, f in enumerate(files):
        unpack_bz2(join(plans_dir, f), source_dir)
        start_time = time.time()
        result = run_experiment(
            obs_file=join(source_dir, "obs.dat"),
            goals_dict_file=None,
            actions_dict_file=None,
            possible_goals_file=join(source_dir, "hyps.dat"),
            correct_goal_file=join(source_dir, "real_hyp.dat"),
            domain=C.SATELLITE,
            verbose=0,
            model_type=C.PERCENTAGE,
            percentage=2,
        )
        exec_time = time.time() - start_time
        if result[0]:
            correct += 1
        total += 1
        times.append(exec_time)
        results_file[0].append(result[1])
        results_file[1].append(result[2])
    results.append(results_file)

In [300]:
acc = list()
for _ in perc_list:
    acc.append(list())
for i, perc_results in enumerate(results):
    acc[i].append(accuracy_score(perc_results[0], perc_results[1]))
print(acc)

[[0.34523809523809523, 0.5952380952380952, 0.6547619047619048, 0.7261904761904762, 0.6666666666666666, 0.75, 0.7738095238095238], [0.2857142857142857, 0.6904761904761905, 0.6904761904761905, 0.7857142857142857, 0.7976190476190477, 0.8928571428571429, 0.8571428571428571], [0.36904761904761907, 0.7142857142857143, 0.7261904761904762, 0.8809523809523809, 0.9166666666666666, 0.9047619047619048, 0.9166666666666666]]


In [301]:
print(acc[0])

[0.34523809523809523, 0.5952380952380952, 0.6547619047619048, 0.7261904761904762, 0.6666666666666666, 0.75, 0.7738095238095238]


## Running Example

In [115]:
source_dir = "./running_example/"
results = list()
times = list()

result = run_experiment(
    obs_file=join(source_dir, "obs.dat"),
    goals_dict_file=None,
    actions_dict_file=None,
    possible_goals_file=join(source_dir, "hyps.dat"),
    correct_goal_file=join(source_dir, "real_hyp.dat"),
    domain=C.BLOCKSWORLD,
    verbose=3,
)

CLEAR P 168
ONTABLE K 288
ON P K 221
ON D J 208
ON R M 505
ON H R 243
ON J F 499
ON C L 383
ON M O 382
ON T G 395
ON B A 79
ON P C 40
ON S K 357
ON G W 50
ON A E 105
ON K P 462
ON F S 285
ON E H 16
ON O J 139
ON L T 271
CLEAR S 491
ONTABLE W 351
ON P J 315
ON K W 453
ON S C 218
CLEAR T 241
ONTABLE S 60
ON M S 194
ON T C 297
ON I C 503
ON U I 166
ON W J 387
ON D P 292
ON R U 436
ON J S 201
ON G Q 435
ON T A 246
ON N T 270
ON S B 332
ON P R 206
ON E N 264
ON A G 182
ON B D 423
ON C E 227
ONTABLE G 222
ON R D 107
ON D G 148
ON Q M 198
ON C K 464
ON M T 279
ON T S 145
ON W A 76
ON D F 235
ON K D 163
ON S W 193
ON F Q 490
ON A S 202
ON Q W 403
ON N D 327
ON D K 72
ON U A 479
ON H Q 188
ON W N 217
ON O U 87
ON S H 431
ON S J 495
ON O I 37
ON L W 155
ON U G 35
ON G S 93
ON I L 225
ON W H 399
ON H U 223
ON J Q 1
ON Q F 402
ON U F 354
ON E A 3
ON A I 183
ON R E 67
ON F J 153
ON H L 232
ON J D 310
ON L C 132
ON C G 113
ON W S 340
ON S U 189
ON M P 440
ON D R 88
ON G N 302
ON N W 363
ON P H 356
O

## Models info

In [376]:
model_type = C.SMALL
percentage = 0
model_dir = C.MODELS_DIR
for domain in range(6):
    model_file = get_domain_related(
        domain, C.MODEL_FILE, model_type=model_type, percentage=percentage
    )
    model = load_model(join(model_dir, model_file), custom_objects=C.CUSTOM_OBJECTS)
    print(model.summary())
    print(model.get_layer("lstm_layer_0").dropout)
    print(model.get_layer("lstm_layer_0").recurrent_dropout)

Model: "satellite_incremental_50perc_best20trials_lossV1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 28, 117)      3887442     input_1[0][0]                    
__________________________________________________________________________________________________
lstm_layer_0 (LSTM)             (None, 28, 496)      1218176     embedding[0][0]                  
__________________________________________________________________________________________________
attention_weights (AttentionWei (None, 28)           524         lstm_layer_0[0][0]               
___________________________________________________

Model: "depots_multiperc_small_best20trials"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 35)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 35, 200)      2610200     input_1[0][0]                    
__________________________________________________________________________________________________
lstm_layer_0 (LSTM)             (None, 35, 450)      1171800     embedding[0][0]                  
__________________________________________________________________________________________________
attention_weights (AttentionWei (None, 35)           485         lstm_layer_0[0][0]               
________________________________________________________________

## GRNet Buckets Experiment

In [113]:
model_type = C.SMALL
percentage = 0
init_models(model_type=model_type, percentage=percentage)
buckets = 9
perc_list = [0.3, 0.5, 0.7]
results = list()
times = list()
source_dir = "./test_instance_pereira/"
for bucket in range(buckets):
    results_bucket = list()
    for perc in perc_list:
        plans_dir = f"../tasks_probatory_1/{int(perc*100)}"
        files = os.listdir(plans_dir)
        total = 0
        correct = 0
        results_file = [list(), list()]
        for j, f in enumerate(files):
            # print(f)
            if f"b=0{bucket+1}" in f:
                if f.endswith(".zip"):
                    unzip_file(join(plans_dir, f), source_dir)
                elif f.endswith(".bz2"):
                    unpack_bz2(join(plans_dir, f), source_dir)
                start_time = time.time()
                result = run_experiment(
                    obs_file=join(source_dir, "obs.dat"),
                    goals_dict_file=None,
                    actions_dict_file=None,
                    possible_goals_file=join(source_dir, "hyps.dat"),
                    correct_goal_file=join(source_dir, "real_hyp.dat"),
                    domain=C.ZENOTRAVEL,
                    verbose=0,
                    model_type=model_type,
                )
                exec_time = time.time() - start_time
                if result[0]:
                    correct += 1
                total += 1
                times.append(exec_time)
                # print(exec_time)
                results_file[0].append(result[1])
                results_file[1].append(result[2])
        results_bucket.append(results_file)
    print(results_bucket)
    results.append(results_bucket)

PLAN TOO LONG
PLAN TOO LONG
PLAN TOO LONG
PLAN TOO LONG
PLAN TOO LONG
[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 6, 1, 5, 5, 0, 7, 7, 6, 0, 0, 2, 5, 3, 0, 0, 0, 0, 7, 0, 0, 5, 8, 2, 0, 0, 0, 3, 3, 0, 0, 4, 0, 1, 0, 0, 0, 0, 7, 0, 1, 0, 8, 2, 0, 7, 8, 9, 0, 8, 0, 8, 9, 2, 0, 7, 0, 5, 5, 9, 1, 2, 1, 0, 2, 1, 0, 0, 3, 0, 8, 0, 0, 0, 8, 0, 0, 0, 8, 0, 0, 0, 9, 0, 5, 4, 0, 4, 0, 0, 8, 0, 0, 0, 0, 8, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 3, 0, 2, 0, 0, 0

PLAN TOO LONG
PLAN TOO LONG
PLAN TOO LONG
PLAN TOO LONG
PLAN TOO LONG
[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 8, 4, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 4, 0, 0, 0, 4, 0, 0, 0, 0, 0, 9, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 9, 5, 0, 0, 0, 0, 0, 0, 0, 5, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 5, 9, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0

In [114]:
print(results)

[[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 6, 1, 5, 5, 0, 7, 7, 6, 0, 0, 2, 5, 3, 0, 0, 0, 0, 7, 0, 0, 5, 8, 2, 0, 0, 0, 3, 3, 0, 0, 4, 0, 1, 0, 0, 0, 0, 7, 0, 1, 0, 8, 2, 0, 7, 8, 9, 0, 8, 0, 8, 9, 2, 0, 7, 0, 5, 5, 9, 1, 2, 1, 0, 2, 1, 0, 0, 3, 0, 8, 0, 0, 0, 8, 0, 0, 0, 8, 0, 0, 0, 9, 0, 5, 4, 0, 4, 0, 0, 8, 0, 0, 0, 0, 8, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 3, 0, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 0, 7, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 6

In [90]:
results = list()
results.append(
    [
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [8, 1, 0, 8, 8, 8, 7, 8, 8, 8]],
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [8, 8, 8, 8, 8, 8, 8, 8, 8, 4]],
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [8, 8, 8, 8, 8, 8, 8, 8, 8, 8]],
    ]
)
results.append(
    [
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [8, 8, 8, 1, 8, 8, 8, 8, 8, 8]],
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [8, 8, 8, 8, 8, 8, 8, 8, 8, 8]],
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [8, 8, 8, 8, 8, 8, 8, 8, 8, 8]],
    ]
)
results.append(
    [
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [8, 8, 5, 8, 8, 8, 8, 0, 8, 2]],
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [8, 8, 5, 8, 8, 8, 8, 8, 8, 8]],
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [8, 8, 8, 8, 8, 8, 8, 8, 8, 8]],
    ]
)
results.append(
    [
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [8, 8, 8, 8, 7, 8, 8, 8, 8, 2]],
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [8, 8, 8, 8, 8, 8, 8, 8, 8, 8]],
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [8, 8, 8, 8, 8, 8, 8, 8, 8, 8]],
    ]
)
results.append(
    [
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [4, 8, 8, 8, 1, 8, 6, 8, 3, 8]],
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [8, 8, 8, 4, 8, 8, 8, 8, 8, 8]],
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [8, 8, 8, 8, 8, 8, 8, 8, 8, 8]],
    ]
)
results.append(
    [
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [8, 8, 8, 8, 8, 8, 8, 8, 8, 8]],
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [8, 8, 8, 8, 8, 8, 8, 8, 8, 8]],
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [8, 8, 8, 8, 8, 8, 8, 8, 8, 8]],
    ]
)
results.append(
    [
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [8, 8, 4, 8, 8, 8, 8, 8, 8, 4]],
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [7, 8, 8, 8, 8, 8, 8, 8, 8, 8]],
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [8, 8, 8, 8, 8, 8, 8, 8, 8, 8]],
    ]
)
results.append(
    [
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [8, 7, 3, 8, 8, 3, 8, 0, 8, 8]],
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [8, 8, 8, 7, 8, 8, 8, 8, 2, 7]],
        [[8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [8, 8, 8, 8, 8, 3, 8, 8, 8, 7]],
    ]
)

In [153]:
percs = [30, 50, 70]
accs = np.zeros((3, 9))
for i, bucket in enumerate(results):
    print(f"Bucket {i+1}")
    for j, perc_results in enumerate(bucket):
        print(
            f" Perc {percs[j]}:{accuracy_score(perc_results[0], perc_results[1])*100 : .1f}"
        )
        accs[j, i] = accuracy_score(perc_results[0], perc_results[1]) * 100

Bucket 1
 Perc 30: 52.0
 Perc 50: 75.0
 Perc 70: 86.0
Bucket 2
 Perc 30: 67.0
 Perc 50: 84.0
 Perc 70: 95.0
Bucket 3
 Perc 30: 73.0
 Perc 50: 85.0
 Perc 70: 96.0
Bucket 4
 Perc 30: 67.0
 Perc 50: 94.0
 Perc 70: 98.0
Bucket 5
 Perc 30: 79.0
 Perc 50: 85.0
 Perc 70: 96.0
Bucket 6
 Perc 30: 82.0
 Perc 50: 91.0
 Perc 70: 100.0
Bucket 7
 Perc 30: 80.0
 Perc 50: 97.0
 Perc 70: 97.0
Bucket 8
 Perc 30: 85.0
 Perc 50: 94.0
 Perc 70: 99.0
Bucket 9
 Perc 30: 88.0
 Perc 50: 96.0
 Perc 70: 99.0


In [154]:
print("[")
for l in accs:
    a = list(l)
    print(f"{list(a)},")
print("]")

[
[52.0, 67.0, 73.0, 67.0, 79.0, 82.0, 80.0, 85.0, 88.0],
[75.0, 84.0, 85.0, 94.0, 85.0, 91.0, 97.0, 94.0, 96.0],
[86.0, 95.0, 96.0, 98.0, 96.0, 100.0, 97.0, 99.0, 99.0],
]


## Pereira classification report

In [101]:
print(get_class_report("./output_depots_pereira/50"))

                                                                                                                                                                            precision    recall  f1-score   support

{'on crate7 pallet1)', 'on crate0 crate7)', 'on crate4 crate1)', 'on crate2 crate6)', 'on crate5 crate3)', 'on crate1 crate0)', 'on crate3 crate2)', 'on crate6 pallet0)'}       1.00      0.33      0.50         3
                                                                                     {'on crate1 crate2)', 'on crate8 crate5)', 'on crate7 pallet0)', 'on crate3 crate0)'}       0.75      1.00      0.86         3
{'on crate7 pallet0)', 'on crate3 crate4)', 'on crate0 crate3)', 'on crate1 crate5)', 'on crate4 crate2)', 'on crate5 crate7)', 'on crate2 crate1)', 'on crate6 pallet1)'}       1.00      0.67      0.80         3
{'on crate7 pallet0)', 'on crate0 crate7)', 'on crate5 crate1)', 'on crate2 crate6)', 'on crate3 crate2)', 'on crate1 crate0)', 'on crate6 pallet1)', '

/opt/anaconda/anaconda3/envs/mchiari/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda/anaconda3/envs/mchiari/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda/anaconda3/envs/mchiari/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result